In [ ]:
!uv add requests --quiet mitreattack-python langchain langchain-anthropic dotenv

In [ ]:
import requests
from pathlib import Path
from mitreattack.stix20 import MitreAttackData
import json
from IPython.display import JSON
import time
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os
from datetime import datetime

In [ ]:
ATTACK_LATEST = "https://github.com/mitre/cti/releases/download/ATT%26CK-v18.1/enterprise-attack.json"
DATA_PATH = Path("data/enterprise-attack.json")
MODEL="claude-haiku-4-5-20251001"

In [ ]:
load_dotenv(".env")

In [ ]:
# print(os.getenv("ANTHROPIC_API_KEY"))

In [ ]:
if not DATA_PATH.exists():
    DATA_PATH.write_bytes(requests.get(ATTACK_LATEST).content)

In [ ]:
mitre_attack_data = MitreAttackData(str(DATA_PATH))

In [ ]:
techniques = mitre_attack_data.get_techniques(include_subtechniques=True, remove_revoked_deprecated=True)

In [ ]:
processed_techniques = sorted([{
    "id": next((r.external_id for r in t.external_references if r.source_name == "mitre-attack"), None),
    "name": t.name,
    "description": t.description or "",
    "tactics": [p.phase_name for p in getattr(t, "kill_chain_phases", []) if p.kill_chain_name == "mitre-attack"],
    "platforms": getattr(t, "x_mitre_platforms", []) or []
} for t in techniques], key=lambda x: (x["id"].split(".")[0], int(x["id"].split(".")[1]) if "." in x["id"] else 0))

In [ ]:
JSON(processed_techniques[155:165])

In [ ]:
llm = ChatAnthropic(model=MODEL, temperature=0, max_tokens=4096)

SYSTEM_PROMPT = """You are a cybersecurity expert compressing MITRE ATT&CK techniques for efficient LLM consumption.

OUTPUT FORMAT (one line per technique):
ID|Tactics|Name|Keywords|Description|Platforms

RULES:
1. ID: Use exact ATT&CK ID (e.g., T1059.001) from input
2. Tactics: Use abbreviations: REC=Reconnaissance, RD=Resource Development, IA=Initial Access, EX=Execution, PE=Persistence, PRV=Privilege Escalation, DE=Defense Evasion, CA=Credential Access, DIS=Discovery, LM=Lateral Movement, COL=Collection, C2=Command and Control, EXF=Exfiltration, IMP=Impact
3. Name: Exact technique name from input
4. Keywords (3-10 terms): Prioritize most identifiable or unique keywords. Filenames, process names, tool names, command flags, protocols. NO generic terms like adversary/attack/malicious
5. Description (20-40 words): Start with action verb. Focus on WHAT happens technically. NO fluff phrases. Focus on specific or unique attributes of the TTP. Format for maximum LLM effectiveness and token-efficiency
6. Platforms: Use platforms from input in comma seperated list

Output ONLY the compressed lines, no explanations, markdown or extra fomatting."""

In [ ]:
def batch(items, size=10):
    for i in range(0, len(items), size):
        yield items[i:i + size]

batches = list(batch(processed_techniques, 10))
print(f"Created {len(batches)} batches")

In [ ]:
compressed_lines = []
errors = []
for i, batch_data in enumerate(batches):
    print(f"Processing batch {i+1}/{len(batches)}...", end=" ")
    
    try:
        prompt = f"Compress these {len(batch_data)} ATT&CK techniques:\n\n{json.dumps(batch_data, indent=2)}\n\nOutput exactly {len(batch_data)} lines."
        
        response = llm.invoke([
            SystemMessage(content=SYSTEM_PROMPT),
            HumanMessage(content=prompt)
        ])
        
        lines = [f"|{l.strip()}|" for l in response.content.strip().split("\n") if l.strip().startswith("T")]
        compressed_lines.extend(lines)
        print(f"OK ({len(lines)} lines)")
        print(response.usage_metadata)
        
        time.sleep(0.5)  # Rate limiting
        
    except Exception as e:
        print(f"ERROR: {e}")
        errors.append((i, batch_data, str(e)))
print(f"\nComplete: {len(compressed_lines)} techniques compressed, {len(errors)} errors")

In [ ]:
header = f"""# ATT&CK-LLM | Enterprise | v18.1 | Generated {datetime.now().strftime("%Y-%m-%d")}
# Tactics: REC=Reconnaissance, RD=Resource Development, IA=Initial Access, EX=Execution, PE=Persistence, PRV=Privilege Escalation, DE=Defense Evasion, CA=Credential Access, DIS=Discovery, LM=Lateral Movement, COL=Collection, C2=Command and Control, EXF=Exfiltration, IMP=Impact
# Techniques (inc. sub-techniques): {len(compressed_lines)}

|ID|Tactics|Name|Keywords|Description|Platforms|
|---|---|---|---|---|---|
"""

output = header + "\n".join(compressed_lines)

# Preview
print(output[:1000])
print(f"\n... ({len(output)} chars, ~{len(output)//4} tokens)")

In [ ]:
with open(f"output/attack_llm_compressed_{MODEL}.md", "w") as f:
    f.write(output)

print(f"Written to output/attack_llm_compressed_{MODEL}.md")

In [ ]:
with open(f"analysing-attack-skill/resources/attack_techniques.md", "w") as f:
    f.write(output)

print(f"Written to analysing-attack-skill/resources/attack_techniques.md")